# 生成供查询股票列表

In [122]:
import time
import requests
import json
import pandas as pd 
import os

In [85]:
# 获取每日行业板块北上持仓情况
def get_daily_north_datasets():
    time_stamp = time_stamp = int(time.time() * 1000 )
    url = 'http://dcfm.eastmoney.com/em_mutisvcexpandinterface/api/js/get?type=HSGT20_HYTJ_SUM&st=ShareSZ_ZC&sr=-1&token=894050c76af8597a853f5b408b759f5d&p=1&ps=60&filter=(DateType%3D%271%27)&_={}'.format(time_stamp)

    response = requests.get(url=url)
    if response.status_code == 200:
        response_json = response.json()
        if type(response_json) is list and len(response_json) > 0:
            return response_json
        else:
            print('获取数据失败！请重试！')
    else:
        print('获取数据失败！请重试！')


In [86]:
north_data = get_daily_north_datasets()

In [94]:
df = pd.DataFrame(north_data)

In [ ]:
# 排序 

In [ ]:
# 生成图表

In [117]:
# 处理数据
#今日持股 
df['ShareSZ_GZ'] = (df['ShareSZ_GZ']/100000000).round(2).astype('str')+'亿'
df['ShareHold_Chg_BK'] = (df['ShareHold_Chg_BK']*100).round(2).astype('str')+'%'
df['ShareHold_Chg_GZ'] = (df['ShareHold_Chg_GZ']*100).round(2).astype('str')+'%'
# # 增持
df['ShareSZ_ZC'] = (df['ShareSZ_ZC']/100000000).round(3).astype('str')+'亿'
df['ShareHold_ZC_Chg'] = (df['ShareHold_ZC_Chg']*100).round(2).astype('str')+'%'
df['ShareHold_ZC_Chg_BK'] = (df['ShareHold_ZC_Chg_BK']*1000).round(2).astype('str')+'‰'
df['ShareHold_ZC_Chg_GZ'] = (df['ShareHold_ZC_Chg_GZ']*1000).round(2).astype('str')+'‰'

,HYCode,ORIGINALCODE,HYName,HdDate,Zdf,ZC_Count,Count,ShareHold_Chg_GZ,ShareSZ_ZC,ShareHold_ZC_Chg,...,ShareSZ_ZC_Max,ShareSZ_JC_Min,ShareHold_ZC_Max,ShareHold_JC_Min,DateType,ShareHold_Chg_BK,Max_ZB_Name,Max_ZB_Code,Min_ZB_Name,Min_ZB_Code
0,016019,456,家电行业,2021-03-17,0.04,13.0,20.0,14.65%,9.42亿,0.51%,...,0.0,0.0,0.0,0.0,1,11.37%,格力电器,000651,老板电器,002508
1,016022,459,电子元件,2021-03-17,-0.54,55.0,110.0,9.7%,5.629亿,0.46%,...,0.0,0.0,0.0,0.0,1,3.59%,TCL科技,000100,威创股份,002308
2,016015,448,通讯行业,2021-03-17,-0.97,21.0,51.0,1.56%,5.462亿,2.86%,...,0.0,0.0,0.0,0.0,1,1.75%,中兴通讯,000063,海能达,002583
3,016040,537,材料行业,2021-03-17,0.10,14.0,27.0,3.57%,4.142亿,0.92%,...,0.0,0.0,0.0,0.0,1,4.81%,国轩高科,002074,沃尔核材,002130
4,016035,481,汽车行业,2021-03-17,-0.21,18.0,41.0,7.8%,4.098亿,0.41%,...,0.0,0.0,0.0,0.0,1,3.76%,德赛西威,002920,华域汽车,600741
5,016032,478,有色金属,2021-03-17,0.07,22.0,36.0,2.06%,3.935亿,1.5%,...,0.0,0.0,0.0,0.0,1,2.39%,云铝股份,000807,中色股份,000758
6,016043,545,机械行业,2021-03-17,0.05,19.0,44.0,7.36%,3.802亿,0.41%,...,0.0,0.0,0.0,0.0,1,7.11%,埃斯顿,002747,山河智能,002097
7,016038,485,旅游酒店,2021-03-17,-0.42,3.0,7.0,5.14%,2.441亿,0.38%,...,0.0,0.0,0.0,0.0,1,8.8%,中青旅,600138,众信旅游,002707
8,016052,732,贵金属,2021-03-17,1.99,8.0,9.0,0.81%,2.209亿,2.14%,...,0.0,0.0,0.0,0.0,1,2.1%,紫金矿业,601899,赤峰黄金,600988
9,016061,910,专用设备,2021-03-17,0.50,24.0,37.0,2.24%,1.733亿,0.61%,...,0.0,0.0,0.0,0.0,1,3.93%,克来机电,603960,中船应急,300527


In [118]:
filter_cols = ['HYName', 
                'Zdf', 
                'Count', 
                'ShareSZ_GZ',
                'ShareHold_Chg_BK',
                'ShareHold_Chg_GZ',
                'ZC_Count',
                'ShareSZ_ZC',
                'ShareHold_ZC_Chg',
                'ShareHold_ZC_Chg_BK',
                'ShareHold_ZC_Chg_GZ',
                'Max_SZ_Name',
                'Max_ZB_Name',
                'Min_SZ_Name',
                'Min_ZB_Name',
                ]
filter_cols_ch = [['', 
                    '', 
                    '北向资金今日持股',
                    '北向资金今日持股',
                    '北向资金今日持股',
                    '北向资金今日持股',
                    '北向资金今日增持估计',
                    '北向资金今日增持估计',
                    '北向资金今日增持估计',
                    '北向资金今日增持估计',
                    '北向资金今日增持估计',
                    '今日增持最大',
                    '今日增持最大',
                    '今日减持最大股',
                    '今日减持最大股'
],
['行业', 
                    '最新涨跌幅', 
                    '支数',
                    '市值',
                    '占板块比',
                    '占北向资金比',
                    '股票支数',
                    '市值',
                    '市值增幅',
                    '占板块比',
                    '占北向资金比',
                    '市值',
                    '占股本比',
                    '市值',
                    '占股本比'
]]

In [119]:
# 替换索引
df = df[filter_cols]
df.columns = filter_cols_ch

In [138]:
def save_data_to_excel(file_name, df):
    root_path = "excels"
    date = time.strftime("%Y-%m-%d", time.localtime())

    date_path = os.path.join(root_path,date)
    file_path = os.path.join(date_path, file_name+'.xlsx')
    
    #判断文件夹是否存在
    if os.path.exists(root_path) is False:
        os.mkdir(root_path)
    if os.path.exists(date_path) is False:
        os.mkdir(date_path)
    # 保存北上数据
    df.to_excel(file_path)


In [139]:
# 保存
save_data_to_excel('行业资金流入情况',df)